In [3]:
spark

In [2]:
ls

anaconda@  df_ll.csv             metastore_db/  vocs.json
derby.log  kiva_spark_aws.ipynb  scripts/


In [4]:
df_ll = spark.read.csv('kiva/df_ll.csv',header=True)

In [12]:
df_ll.show(1)

+---+-------+-----------+--------+
|_c0|LOAN_ID|LENDERS_lst|lender_v|
+---+-------+-----------+--------+
|  0| 483693|     muc888|  970524|
+---+-------+-----------+--------+
only showing top 1 row



In [15]:
df_ll.select('LOAN_ID','lender_v').show(1)

+-------+--------+
|LOAN_ID|lender_v|
+-------+--------+
| 483693|  970524|
+-------+--------+
only showing top 1 row



In [23]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry

In [5]:
from pyspark.sql.functions import lit

df_with_one = df_ll.withColumn("value", lit(1))
df_with_one.show(4)

+---+-------+--------------+--------+-----+
|_c0|LOAN_ID|   LENDERS_lst|lender_v|value|
+---+-------+--------------+--------+-----+
|  0| 483693|        muc888|  970524|    1|
|  1| 483693|       sam4326| 1153379|    1|
|  2| 483693|   camaran3922|  187221|    1|
|  3| 483693|   lachheb1865|  758772|    1|
|  4| 483693|   rebecca3499| 1090160|    1|
|  5| 483693| karlheinz4543|  687280|    1|
|  6| 483693|       jerrydb|  588362|    1|
|  7| 483693|     paula8951| 1043143|    1|
|  8| 483693|          gmct|  460491|    1|
|  9| 483693|      amra9383|   55499|    1|
| 10| 483693|         r3922| 1070591|    1|
| 11| 483693|     brian9451|  173534|    1|
| 12| 483693|     shree8053| 1211636|    1|
| 13| 483693|      alan5513|   20910|    1|
| 14| 483693|     oisin3389| 1013325|    1|
| 15| 483693|     helle8622|  493827|    1|
| 16| 483693|        bo3186|  155244|    1|
| 17| 483693|       ric8947| 1100676|    1|
| 18| 483693|daniel98469874|  293820|    1|
| 19| 483693|      nick9464|  99

In [28]:
mat = CoordinateMatrix(df_with_one.select('LOAN_ID','lender_v','value').rdd.map(lambda r: MatrixEntry(r.LOAN_ID, r.lender_v,r.value)))

In [6]:
ratings = df_with_one.select('LOAN_ID','lender_v','value')

In [36]:
type(ratings)

pyspark.sql.dataframe.DataFrame

In [7]:
ratings.show(4)

+-------+--------+-----+
|LOAN_ID|lender_v|value|
+-------+--------+-----+
| 483693|  970524|    1|
| 483693| 1153379|    1|
| 483693|  187221|    1|
| 483693|  758772|    1|
+-------+--------+-----+
only showing top 4 rows



In [10]:
splits = ratings.randomSplit([1.0, 2.0], 17)

In [12]:
splits[0].show(2)

+-------+--------+-----+
|LOAN_ID|lender_v|value|
+-------+--------+-----+
|   1000|  308811|    1|
|   1000|  579129|    1|
+-------+--------+-----+
only showing top 2 rows



In [14]:
splits[0].count()

9436011

In [15]:
splits[1].count()

18858843

In [8]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [16]:
rank = 10
numIterations = 10
model = ALS.train(splits[1], rank, numIterations)

In [ ]:
# splits[0]..map(lambda r: ((r[0], r[1]), r[2]))

In [22]:
predictions = model.predictAll(splits[0].rdd.map(lambda r: (r[0], r[1]), r[2])))

SyntaxError: unexpected EOF while parsing (<ipython-input-22-2ac54626ffc6>, line 1)

In [ ]:
model2 = ALS.trainImplicit(ratings, rank, numIterations, alpha=0.01)